# 可迭代的对象、迭代器和生成器

## Sentence类第1版：单词序列



In [4]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [5]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


序列可以迭代的原因： iter函数

它有如下作用：
+ 检查对象是否实现了 `__iter__` 方法，如果实现了就调用，获取一个迭代器。
+ 如果没有实现`__iter__`方法，但是实现了`__getitem__`方法，python会创建一个迭代器，尝试按顺序获取元素
+ 如果尝试失败，Python会抛出`TypeError`异常，通常会提示'C object is not iterable'（C对象不可迭代），其中C是目标对象所属的类。


这是鸭子类型的极端形式，不仅要实现特殊的`__iter__`方法，还要实现`__getitem__`方法，而且`__`；

在白鹅类型理论中，可迭代对象的定义简单一些，不过没有那么灵活；如果实现了 `__iter__`方法，那么就认为对象是可迭代的。此时不需要创建子类;
也不用注册，因为`abc.Iterable`类实现了`__subclasshook__`方法。

## 可迭代的对象与迭代器的对比

使用iter内置函数可以获取迭代器的对象，如果对象实现了能返回迭代器的`__iter__`方法，那么对象就是可迭代的，序列都可以迭代，实现了`__getitem__`方法，而且其参数是从零开始的索引，这种对象也可以迭代。

In [7]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [8]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


+ 迭代器:
   迭代器是这样的对象，实现了无参数的`__next__`方法，返回序列中的下一个元素：如果没有元素了，那么抛出StopIteration。Python中的迭代器还实现了`__iter__`方法，因此迭代器也可以迭代。
   因为内置的iter函数会对序列做特殊处理，所以第1版Sentence类可以迭代
   
接下来需要实现典型的迭代器


## Sentence类第2类：典型的迭代器



In [ ]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
            
        self.index += 1
        return word
    
    def __iter__(self):
        return self